## 작업형 제 3 유형

### ✏️ [로지스틱 회귀] 조개 정보를 나타낸 데이터다. 주어진 데이터에서 300개 중 앞에서부터 210개에는 train 으로, 90개는 test 데이터로 나눈다. 모델을 학습(적합)할 때는 train 데이터를 사용하고, 예측할 때는 ㅅest 데이터를 사용한다. 모델은 다중 로지스틱 회귀를 써서 성별(gender)을 예측하되, 패널티는 부과하지 않는다

- 종속변수 : gender(0: 암컷, 1: 수컷)

** 패널티는 주로 L1패널티(Lasso 회귀), L2패널티(Ridge 회귀) 등이 있는데 과적합을 방지하기 위해 사용됨

<br>

#### [문제1-1] weight 를 독립변수, gender 를 종속변수로 사용해 로지스틱 회귀 모델을 만들고, weight 변수가 무게가 한 단위 증가할 때 수컷일 오즈비 값은? (반올림하여 소수 넷째 자리까지 계산)

#### [문제1-2] gender 를 종속변수로 하고 나머지 변수돌(age, length, diameter, height, weight)을 독립변수로 사용하는 로지스틱 회귀 모델을 적합시킨 후 잔차 이탈도(Residual Deviance)를 계산하시오. (반올림하여 소수 둘째 자리까지 계산)

#### [문제1-3] 독립변수 weight 만 사용해 학습한 모델에서 test 데이터의 gender 를 예측하고, Error Rage(오류율)를 구하시오. (반올림하여 소수 셋째 자리까지 계산)

In [1]:
import pandas as pd
df = pd.read_csv('./data/clam.csv')
print(df.head(), '\n')

print(df.shape)

   age    length  diameter    height     weight  gender
0    6  0.474627  0.211352  0.178189  78.971766       1
1    1  0.465847  0.339388  0.170522  98.781960       1
2    4  0.122807  0.238691  0.106924  88.792625       0
3    4  0.204579  0.360543  0.034261   1.028847       0
4    8  0.243458  0.358037  0.128080   6.503367       0 

(300, 6)


In [2]:
# 데이터 분할
train = df.iloc[:210]
test =df.iloc[210:]
print(train.shape, '\n')
print(train.head())

(210, 6) 

   age    length  diameter    height     weight  gender
0    6  0.474627  0.211352  0.178189  78.971766       1
1    1  0.465847  0.339388  0.170522  98.781960       1
2    4  0.122807  0.238691  0.106924  88.792625       0
3    4  0.204579  0.360543  0.034261   1.028847       0
4    8  0.243458  0.358037  0.128080   6.503367       0


In [3]:
# [문제1-1]
# 1. 로지스틱 회귀 모델 생성 및 학습
## logit() 함수를 사용해 로지스틱 회귀 모델을 생성하고 학습
## 모델은 gender 를 종속변수, weight 를 독립변수로 사용
from statsmodels.formula.api import logit
import numpy as np

model = logit('gender ~ weight', data=train).fit()

# 2. 오즈비 계산
## 학습된 모델에서 weight 에 대한 계수 값 가져온 후 np.exp() 함수를 사용해 오즈비로 변환
## round() 함수를 사용해 오즈비를 소수점 넷째 자리까지 반올림하여 출력
## 계산된 오즈비는 weight 가 한 단위 증가할 때마다 gender 가 수컷일 확률이 얼마나 증가하는지를 나타냄
odds_ratio = np.exp(model.params['weight'])
print(round(odds_ratio, 4))

Optimization terminated successfully.
         Current function value: 0.690045
         Iterations 4
1.0047


In [4]:
# [문제1-2]
# 1. 로지스틱 회귀 모델 생성 및 학습
## 문제에서 요구한 독립변수를 사용해 로지스틱 회귀 모델을 적합(학습)
from statsmodels.formula.api import logit
formula = 'gender ~ age + length + diameter + height + weight'
model = logit(formula, data=train).fit()

# 2. 잔차이탈도
## 잔차이탈도(residual deviance)는 모델의 적합도를 평가하는 지표, 로그 우도 값에 -2 를 곱한 값
## 로그 우도(log-likelihood) 값이 클수록 모델이 데이터를 잘 설명한다는 뜻
### 로지스틱 회귀 모델의 로그 우도는 모델이 주어진 데이터에 얼마나 잘 맞는지를 나타내는 지표
print(round(-2 * model.llf, 2))

Optimization terminated successfully.
         Current function value: 0.683173
         Iterations 4
286.93


In [5]:
# [문제1-3]
# 1. test 데이터를 사용해 예측(0.5 미만: 0, 0.5 이상: 1)
## 모델을 학습할 때는 train 데이터를 사용하고, 예측할 때는 test 데이터 사용
## gender 를 target 변수에 옮겨두고 성별(gender)을 예측
## 로지스틱 회귀 모델(logit)을 통해 각 관측치가 1에 속할 확률 계산
## 이 확률이 0.5 이상이면 1로 분류, 0.5 미만이면 0으로 분류
## 이렇게 해서 실제 범주형 예측값(0 또는 1)이 저장됨
from sklearn.metrics import accuracy_score

## 로지스틱 회귀 모델 생성 및 학습 (독립변수 weight 만 사용)
model = logit('gender ~ weight', data=train).fit()

target = test.pop('gender')
pred = model.predict(test)
pred = (pred > 0.5).astype(int)

# 2. 실제 값과 예측값을 사용해 정확도 계산
## accuracy_score() 함수는 실제값(target)과 예측값(pred)을 받아 모델의 정확도 계산
## 정확도는 예측 중에서 올바르게 예측된 비율을 의미
accuracy = accuracy_score(target, pred)

# 3. 오류율 계산
## 오류율은 1에서 정확도를 뺀 값으로 잘못 예측된 비율을 나타냄
error_rate = 1 - accuracy
print(round(error_rate, 3))

Optimization terminated successfully.
         Current function value: 0.690045
         Iterations 4
0.478


---

#### 💡 Statsmodels 로지스틱 회귀 주요 속성 및 메서드

|속성/메서드|설명|
|:-:|-|
|model.params|회귀 계수 (Coefficients)|
|model.pvalues|각 독립변수에 대한 p-value|
|model.summary()|모델의 요약 결과표 (Coefficients, Pseudo R-squared 등 포함)|
|model.llf|모델의 로그-우도 (Log-Likelihood of the fitted model)|
|model.llnull|Null 모델의 로그-우도|
|model.bic|Bayesian Information Criterion (BIC)|
|model.aic|Akaike Information Criterion (AIC)|
|model.df_model|사용된 독립변수의 개수|
|model.df_resid|자유도 잔차 (Residual degrees of freedom)|
|model.pred_table()|Confusion Matrix (예측 vs 실제값)|
|model.fittedvalues|학습 데이터에 대한 예측값 (확률)|
|model.resid_dev|잔차 이탈도 (Deviance Residuals)|
|model.bse|회귀 계수의 표준 오차 (Standard Errors)|

<br>

---

#### 💡 오류율(Error Rate)
- 분류 모델에서 예측한 결과가 실제 값과 얼마나 다른지를 나타내는 지표

    - 모든 예측 중에서 잘못된 예측의 비율

- 오류율 = 잘못된 예측의 수 / 전체 예측의 수

- 정확도(Accuracy) : 모든 예측 중 올바른 예측의 비율

    - 오류율과 정확도는 서로 보완적인 관계

    - 오류율이 낮을 수록 모델의 성능이 좋다고 평가

        - 정확도 = 1 - 오류율

---

### ✏️ [선형 회귀]

#### [문제2-1] ERP 와 가장 상관 관계가 높은 값을 구하시오. (반올림하여 소수 셋째 자리까지 계산)

#### [문제2-2] CPU 컬럼이 100 미만인 것만 찾아 ERP 를 종속변수, 나머지 변수들을 독립변수로 설정해 선형 회귀 모델을 만들고 적합한 결정 계수를 구하시오. (반올림하여 소수 셋째 자리까지 계산)

#### [문제2-3] 문제 2-2에서 만든 모델에서 독립변수 중 p-value 가 가장 높은 값을 구하시오. (반올림하여 소수 셋째 자리까지 계산)

In [6]:
import pandas as pd
df = pd.read_csv('./data/system_cpu.csv')
print(df.head())

     ERP  Feature1  Feature2  Feature3    CPU
0   30.6     235.1      44.5      44.0  112.3
1   40.3      36.6      46.4      36.1   58.6
2   57.7      52.2      66.5       2.0   55.3
3  128.3     196.2      59.8      57.4  103.2
4   80.3      75.2      59.6      58.2  104.1


In [7]:
# 문제[2-1]
# 1. ERP 와 각 변수 사이의 상관계수 계산
## 데이터프레임의 corr()를 사용해 모든 변수 간의 상관 계수 계산
corr_matrix = df.corr()
print(corr_matrix, '\n')

# 2. ERP 와 다른 변수들과의 상관계수 출력
## ERP 변수와 다른 모든 변수들과의 상관 계수를 선택하고, 이를 내림차순(ascending=False)으로 정렬
##  - ERP 와 가장 높은 상관 관계를 가진 변수부터 시작해 낮은 상관 계수를 가진 변수 순으로 정렬
## 상관 계수의 절대값 중요
##  - ex. 상관 계수가 +0.9 와 -0.95 인 두 변수가 있다면, -0.95 의 절댓값이 더 크기 때문에 음의 상관 계수를 가진 변수가 ERP 와 더 강한 상관 관계를 가짓 것으로 간주
erp_corr = corr_matrix['ERP'].sort_values(ascending=False)
print(erp_corr, '\n')
print('정답 :', round(erp_corr.values[1], 3))

               ERP  Feature1  Feature2  Feature3       CPU
ERP       1.000000 -0.053848  0.092432  0.882194  0.092455
Feature1 -0.053848  1.000000 -0.020095 -0.015767  0.023664
Feature2  0.092432 -0.020095  1.000000  0.118877  0.130492
Feature3  0.882194 -0.015767  0.118877  1.000000  0.137120
CPU       0.092455  0.023664  0.130492  0.137120  1.000000 

ERP         1.000000
Feature3    0.882194
CPU         0.092455
Feature2    0.092432
Feature1   -0.053848
Name: ERP, dtype: float64 

정답 : 0.882


In [9]:
# [문제2-2]
# 1. 
cond = df['CPU'] < 100
filtered_df = df[cond]

# 2. 
from statsmodels.formula.api import ols

model = ols('ERP ~ Feature1 + Feature2 + Feature3 + CPU', data=filtered_df).fit()

# 
print(model.summary())
print('정답 :', round(model.rsquared, 3))

                            OLS Regression Results                            
Dep. Variable:                    ERP   R-squared:                       0.755
Model:                            OLS   Adj. R-squared:                  0.736
Method:                 Least Squares   F-statistic:                     39.30
Date:                Thu, 28 Nov 2024   Prob (F-statistic):           5.36e-15
Time:                        17:33:02   Log-Likelihood:                -260.40
No. Observations:                  56   AIC:                             530.8
Df Residuals:                      51   BIC:                             540.9
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     51.4133     19.112      2.690      0.0

---

#### 💡 Statsmodels 선형 회귀 주요 속성 및 메서드

|속성/메서드|설명|
|:-:|-|
|model.params|회귀 계수 (Coefficients)|
|model.pvalues|각 독립변수에 대한 p-value|
|model.rsquared|결정 계수 (R-squared, 모델 적합도)|
|model.rsquared_adj|조정된 결정 계수 (Adjusted R-squared)|
|model.summary()|모델 요약 결과표 (Coefficients, R-squared 등 포함)|
|model.fvalue|F-statistic 값 (모델 전체 유의성 테스트)|
|model.f_pvalue|F-statistic에 대한 p-value|
|model.aic|Akaike Information Criterion (AIC)|
|model.bic|Bayesian Information Criterion (BIC)|
|model.resid|잔차 (Residuals)|
|model.fittedvalues|학습 데이터에 대한 예측값|
|model.predict()|새로운 데이터에 대한 예측|

<br>

---

In [11]:
# [문제2-3]
# 1. p-value 에서 최대값과 해당 변수명 찾기
max_pvalue = model.pvalues.max()    # 가장 높은 p-value
max_pvalue_variable = model.pvalues.idxmax()    # 가장 높은 p-value 를 가진 변수명

# 2. 결과 출력
print('가장 높은 p-value 를 가진 변수명 :', max_pvalue_variable)
print('정답(가장 높은 p-value) :', round(max_pvalue, 3))

가장 높은 p-value 를 가진 변수명 : Feature1
정답(가장 높은 p-value) : 0.684
